![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Compiling Cactus!
Step 1 is to download the code. Cactus uses a script named "GetComponents" to find and prepare all the source code modules that it needs for a given installation. The GetComponents script can be downloaded with a simple invocation of curl.

In [ ]:
%cd ~/
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2016_11/GetComponents

Step 2 is to download your thornlist. In this tutorial, we are going to use Funwave, a collection of thorns
designed to simulate water waves using the Boussinesq equations.

In [ ]:
!curl -kLO https://bitbucket.org/stevenrbrandt/cajunwave/raw/master/funwave_carpet.th

You can view a file in the notebook by using the "magic" command "%pycat filename". However, it tries to highlight
syntax as if the file is written in python. In those cases you can simply use "%cat filename." Unfortunately, unlike %pycat, %cat leaves the contents of the file on the screen.

Note that at the top of the file is "DEFINE_ROOT = CactusFW2". This means that Cactus, and all its thorns, will be checked out under that directory.

In [ ]:
%pycat ~/funwave_carpet.th

Next we need to checkout the components listed in the thornlist. We do this with the GetComponents command.
Before we can execute it, however, we need to turn on its execute bit.

In [ ]:
%cd ~/
!chmod a+x GetComponents
!echo no|./GetComponents --parallel funwave_carpet.th

In [ ]:
%cd ~/CactusFW2

<img src="http://simfactory.org/global/images/factory.png" width="200px">

# Simfactory
Cactus is normally built with a tool called Simfactory. Simfactory, in turn, will call make.
Before it can work, however, it needs to be configured. Please replace... my email address in the
command below with yours. The email address isn't sent anywhere, all it's used for is allowing
Cactus to send job change state notifications to you.

In [ ]:
%cd ~/CactusFW2
!./simfactory/bin/sim setup-silent --setup-email=sbrandt@cct.lsu.edu 

At long last, we are ready to actually build Cactus. Cactus can often figure out what compilers and build
options to use automatically, but in some cases it is necessary to specify it by hand (you can do this by adding --optionlist=centos.cfg to the build command below). The file containing
this information is called the Option List. You might want to take a look at it.

This is the command to build Cactus using our thornlist. As written, it will build in parallel using two processes. That's what the -j option does.

In [ ]:
#!rm -fr configs

In [ ]:
!time ./simfactory/bin/sim build --mdbkey make 'make -j2' --thornlist=./repos/cajunwave/funwave_carpet.th | cat -

The build command creates a configuration called "sim". It is found in the "configs/sim" directory. One of the files in this directory is the ThornList. It contains the list of thorns Cactus will compile. If you wish to add or remove a thorn from your configuration, you can do it by editing this file. However, by doing so you risk confusing yourself by forgetting what you've done. Proceed at your own risk!

In [ ]:
%ls ~/CactusFW2/configs/sim

In [ ]:
%pycat ~/CactusFW2/configs/sim/ThornList

The "OptionList" file contains all the configuration options (the things you saw in centos.cfg). Unlike the ThornList file, however, changing this file will have no effect. If you wish to change your configuration options without starting over from scratch, you should edit the file "configs/sim/config-data/make.config.defn."

In [ ]:
%pycat ~/CactusFW2/configs/sim/config-data/make.config.defn

<h1>Running Cactus!</h1>

In [ ]:
%cd ~/CactusFW2

Below we are going to run a simple Gaussian water wave over a flat seabed. We will use MPI and run on two processes. You can edit the parameter file below and hit shift-Enter to write it to disk. The special sequence "%%writefile filename" at the top makes this possible. Alternatively, you can load an existing file by putting the special sequence "%load filename" at the top of a cell and hitting shift-Enter.

In [ ]:
%%writefile ~/CactusFW2/wave.par

#Reorder the parameters for easy comparison to the input.txt in example 3
ActiveThorns = "
  CoordBase FunWave FunwaveCoord CartGrid3D Carpet CarpetIOASCII
  CartGrid3D IOUtil CarpetIOBasic CarpetSlab Boundary SymBase MoL
  CarpetReduce LocalReduce InitBase CarpetLib LoopControl Tridiagonal
  CarpetIOScalar "

#----------------------------------------------------
# Flesh and CCTK parameters
#----------------------------------------------------

# flesh
Cactus::cctk_run_title = "Test Run"
Cactus::cctk_show_schedule = "yes"
Cactus::cctk_itlast = 300
Cactus::allow_mixeddim_gfs = "yes"

# CartGrid3D
CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin =  0
CoordBase::xmax =  30
CoordBase::ymin =  0
CoordBase::ymax =  30
CoordBase::zmin =  0.0
CoordBase::zmax =  0.0
CoordBase::dx   =  0.25
CoordBase::dy   =  0.25

CoordBase::boundary_size_x_lower     = 3
CoordBase::boundary_size_x_upper     = 3
CoordBase::boundary_size_y_lower     = 3
CoordBase::boundary_size_y_upper     = 3
CoordBase::boundary_size_z_lower     = 0
CoordBase::boundary_size_z_upper     = 0
CoordBase::boundary_shiftout_x_lower = 1
CoordBase::boundary_shiftout_x_upper = 1
CoordBase::boundary_shiftout_y_lower = 1
CoordBase::boundary_shiftout_y_upper = 1
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

# Carpet
Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 3
Carpet::ghost_size_y = 3
Carpet::ghost_size_z = 1
carpet::adaptive_stepsize = yes

# MoL
MoL::ODE_Method = "RK3"
MoL::disable_prolongation        = "yes"

# the output dir will be named after the parameter file name
IO::out_dir = $parfile
IO::out_fileinfo="none"
IOBasic::outInfo_every = 1
IOBasic::outInfo_vars = "FunWave::eta FunWave::u FunWave::v"

#IOASCII::out1D_every = 1
#IOASCII::out1d_vars = "FunWave::eta Funwave::depth"
CarpetIOASCII::compact_format = false
IOASCII::out2D_every = 30
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "FunWave::eta FunWave::u FunWave::v"
IOASCII::out2D_xz = "no"
IOASCII::out2D_yz = "no"
IOASCII::output_ghost_points = "no"

IOScalar::outScalar_every = 1
IOScalar::outScalar_vars = "FunWave::eta FunWave::u FunWave::v"

#& = "Funwave::eta"

#----------------------------------------------------
# Funwave parameters
#----------------------------------------------------

# Funwave depth 
FunWave::depth_file_offset_x = 3
FunWave::depth_file_offset_y = 3
FunWave::depth_type = "flat"
FunWave::depth_format = "ele"
FunWave::depth_file = "/tmp/__depth__.txt"
FunWave::depth_flat = 0.8
#Funwave::test_depth_shore_x = 80
#Funwave::test_depth_island_x = 40
#Funwave::test_depth_island_y = 40
FunWave::depth_xslp = 10.0
FunWave::depth_slope = 0.05
FunWave::dt_size = 0
Funwave::generate_test_depth_data = true
Funwave::num_wave_components = 1
Funwave::wave_component_file = "/home/sbrandt/workspace/shi_funwave/example_2/fft/wavemk_per_amp_pha.txt"
Funwave::peak_period = 1

# import
Funwave::time_ramp = 1.0
Funwave::delta_wk = 0.5
Funwave::dep_wk = 0.45
Funwave::xc_wk = 3.0
Funwave::ywidth_wk = 10000.0
Funwave::tperiod = 1.0
Funwave::amp_wk = 0.0232
Funwave::theta_wk = 0.0
Funwave::freqpeak = 0.2
Funwave::freqmin = 0.1
Funwave::freqmax = 0.4
Funwave::hmo = 1.0
Funwave::gammatma = 5.0
Funwave::thetapeak = 10.0
Funwave::sigma_theta = 15.0

# Funwave wind forcing
Funwave::wind_force = false
Funwave::use_wind_mask = false
Funwave::num_time_wind_data = 2
Funwave::timewind[0] = 0
Funwave::wu[0] = 25
Funwave::wv[0] = 50
Funwave::timewind[1] = 1000
Funwave::wu[1] = 100
Funwave::wv[1] = 100
Funwave::boundary = funwave

# Funwave wave maker
FunWave::wavemaker_type = "ini_gau"
FunWave::xc = 26.5
FunWave::yc = 26.9
FunWave::amp =  2.0
FunWave::wid =  1
Funwave::wdep = 0.78
Funwave::xwavemaker = 25.0

# Funwave sponge 
FunWave::sponge_on = false
FunWave::sponge_west_width = 2.0
FunWave::sponge_east_width = 2.0
FunWave::sponge_north_width = 0.0
FunWave::sponge_south_width = 0.0
FunWave::sponge_decay_rate = 0.9
FunWave::sponge_damping_magnitude = 5.0

# Funwave dispersion (example 3 enables dispersion)
FunWave::dispersion_on = "true"
FunWave::gamma1 = 1.0
FunWave::gamma2 = 1.0
FunWave::gamma3 = 1.0
FunWave::beta_ref = -0.531
FunWave::swe_eta_dep = 0.80
FunWave::cd = 0.0

# Funwave numerics (MoL parameter controls time integration scheme)
FunWave::reconstruction_scheme = "fourth"
FunWave::riemann_solver = "HLLC"
FunWave::dtfac = 0.5
FunWave::froudecap = 10.0
FunWave::mindepth = 0.001
FunWave::mindepthfrc = 0.001
FunWave::enable_masks = "true"
Funwave::estimate_dt_on = "true"

FunwaveCoord::spherical_coordinates = false

ActiveThorns = "CarpetIOHDF5"
IOHDF5::out2D_xyplane_z = 0 
IOHDF5::out2D_every = 10
IOHDF5::out2D_vars = " 
  FunWave::eta
  FunWave::u
  FunWave::v
  Grid::Coordinates{out_every=1000000000}
"
IOHDF5::out2D_xz = no
IOHDF5::out2D_yz = no

This next cell deletes our simulation in case we want to throw it away and start over again for some reason.

In [ ]:
!rm -fr ~/simulations/wave

At long last, we are ready run Cactus. This configuration specifies running on two threads, with 1 thread per process. To execute this command, Cactus uses a "RunScript" stored in configs/sim/RunScript. You might want to take a look at it. Identifiers sandwiched between @ symbols get replaced by Simfactory prior to execution.

In [ ]:
!cat ~/CactusFW2/configs/sim/RunScript

Enough already! Let's run Cactus!

In [ ]:
%cd ~/CactusFW2
!./simfactory/bin/sim create-run --procs 2 --num-threads 1 wave.par

Data can be found in this directory. Using the next couple of commands, we will browse it.

In [ ]:
%cd ~/simulations/wave/output-0000/wave

In [ ]:
%ls *.asc

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

The top of %pycat command showed us what the columns mean:
HYDROBASE::rho (hydrobase-rho)
* 1 iteration
* 2 time - how much time has passed in the simulation
* 3 the data, in this case the variable rho

Once we know all this, it is straightforward to plot the data.

In [ ]:
lin_data = np.genfromtxt("eta.maximum.asc")

In [ ]:
lin_data

In [ ]:
plt.plot(lin_data[:,1],lin_data[:,2])

Python knows how to read regularly formatted text
files that use the # character for comments. Fortunately,
that's what Cactus produces in its asc files.

In [ ]:
file_data = np.genfromtxt("eta.xy.asc")

In [ ]:
file_data

In [ ]:
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.gist_rainbow

In [ ]:
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,12]),np.max(file_data[:,12])
for which in sets: 
    print("which=",which)
    g = file_data[file_data[:,0]==which,:]
    x = g[:,5]
    y = g[:,6]
    z = g[:,12]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

In [ ]:
%ls *.h5

<h2>Plotting HDF5 Data</h2>
HDF5 (Hierarchical Data Format 5) is a portable binary data format. As such, it is far more efficient to read and
write than ascii formats, and it is probably what you should normally use. Here, you can see how to read and display
the data.

In [ ]:
import h5py

In [ ]:
f5 = h5py.File("u.xy.h5")
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    print("nm=",nm)
    d=np.copy(f5[nm])
    plt.figure()
    plt.imshow(d[::-1,:])
    plt.show()

Unfortunately, each set only has one component of the plot, i.e. the part belonging to one processor. To fix this, we'll
collect data sets belonging to an iteration and display them all together. In order to make this happen, we'll need the
x and y values for each component of the grid.

In [ ]:
import re

In [ ]:
f5x = h5py.File("x.xy.h5")
f5y = h5py.File("y.xy.h5")
x_coords = {}
y_coords = {}
for nm in f5x:
    print(nm)
    m = re.search(r'rl=.*c=\d+',nm)
    if m:
        k = m.group(0)
        x_coords[k]=np.copy(f5x[nm])
for nm in f5y:
    m = re.search(r'rl=.*c=\d+',nm)
    if m:
        k = m.group(0)
        y_coords[k]=np.copy(f5y[nm])

In [ ]:
f5 = h5py.File("u.xy.h5")
mn,mx = None,None

# Compute the min and max
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    d5 = np.copy(f5[nm])
    tmin = np.min(d5)
    tmax = np.max(d5)
    if mn == None:
        mn,mx = tmin,tmax
    else:
        if tmin < mn:
            mn = tmin
        if tmax > mx:
            mx = tmax
            
# Collect all the pieces into the d5_tl dictionary
d5_tl = {}            
for nm in f5:
    if not hasattr(f5[nm],"shape"):
        continue
    # Parse the string nm...
    m = re.search(r'it=(\d+)\s+tl=\d+\s+(rl=(\d+)\s+c=(\d+))',nm)
    # group(1) is the iteration number
    # group(2) is "rl={number} c={number}"
    # group(3) is the number in "rl={number}"
    # group(4) is the number in "c={number}"
    grid = int(m.group(1))
    comp = int(m.group(4))
    k = m.group(2)
    if grid in d5_tl:
        d5_tl[grid]["x"] += [x_coords[k]] # append to the x array
        d5_tl[grid]["y"] += [y_coords[k]] # append to the y array
        d5_tl[grid]["D"] += [f5[nm]] # append to the data array
    else:
        d5_tl[grid] = {
            "x":[x_coords[k]],
            "y":[y_coords[k]],
            "D":[f5[nm]]
        }

# Sort the keys so that we display time levels in order
def keysetf(d):
    a = [] # create an empty list
    for k in d: # for each key in d
        a.append(k) # append it to the list
    return a
kys = keysetf(d5_tl.keys())
kys.sort()

# Show the figures, combing data from the same time level
for index in kys:
    data = d5_tl[index]
    print("iteration=",index)
    plt.figure() # put this before the plots you wish to combine
    plt.pcolor(data["x"][0],data["y"][0],data["D"][0],vmin=mn,vmax=mx)
    plt.pcolor(data["x"][1],data["y"][1],data["D"][1],vmin=mn,vmax=mx)
    plt.show() # show the plot.

<h3>Questions and Exercises:</h3>

* Run the above simulation using a single process instead of two. Do the plotting routines work? What changes did you have to make. What would you need to do to make it work with 3?
* Run the code at 1/2 the resolution.
* Position the Guassian wave at a different place on the grid.
* If you wanted to change the compiler or a compiler flag, how would you go about doing that?
* If you wanted to add another thorn to the list of thorns to compile, how would you go about doing that?
* If you wanted to create a thornlist that would check out Cactus under the Foo directory instead of the CactusFW2 directory, how would you do it?

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a> and <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"> CCF 1539567</a></td><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>